In [1]:
!pip install tensorflow
!pip install matplotlib
!pip install ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.3 MB/s eta 0:00:00


In [2]:
# Import necessary libraries
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageFilter
import io
import ipywidgets as widgets


# Introduction
This lab is designed to introduce you to the basics of deep learning by interacting with a pre-built model. You'll understand the workflow of a deep learning project, including data preprocessing, model architecture, and making predictions. The goal is to familiarize yourself with the basics of deep learning without writing any code.


In [3]:
# Load the VGG16 model
model = VGG16(weights='imagenet')

# Display the model architecture
model.summary()


553467096/553467096 ━━━━━━━━━━━━━━━━━━━━ 9s 0us/step


Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ fc1 (Dense)                          │ (None, 4096)                │     102,764,544 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 138,357,544 (527.79 MB)

 Trainable params: 138,357,544 (527.79 MB)

 Non-trainable params: 0 (0.00 B)

In [19]:
# Load and preprocess an image
def load_and_preprocess_image(image_path):
    # Load the image
    import os # Corrected indentation
    current_directory = os.getcwd() # Corrected indentation

    # Check if 'images' folder exists and create it if it doesn't
    images_folder = os.path.join(current_directory, 'images')
    if not os.path.exists(images_folder):
        os.makedirs(images_folder)
        print(f"Created 'images' folder at {images_folder}")

    # Build the full path to the image
    image_path = os.path.join(images_folder, 'sample.jpg')

    # Check if 'sample.jpg' exists and suggest how to upload if it doesn't
    if not os.path.exists(image_path):
        print(f"Image file 'sample.jpg' not found at {image_path}")
        print("Please upload the 'sample.jpg' file to the 'images' folder")
        return None, None  # Return None if image is not found

    # Proceed with loading the image
    img = load_img(image_path, target_size=(224, 224)) # Load image using load_img

    # ... (rest of your function remains the same)

In [21]:
# Import necessary libraries
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageFilter
import io
import ipywidgets as widgets

# Load the VGG16 model
model = VGG16(weights='imagenet')

# Display the model architecture
#model.summary()

# Load and preprocess an image
def load_and_preprocess_image(image_path):
    # Load the image
    import os
    current_directory = os.getcwd()

    # Check if 'images' folder exists and create it if it doesn't
    images_folder = os.path.join(current_directory, 'images')
    if not os.path.exists(images_folder):
        os.makedirs(images_folder)
        print(f"Created 'images' folder at {images_folder}")

    # Build the full path to the image
    image_path = os.path.join(images_folder, 'sample.jpg')

    # Check if 'sample.jpg' exists and suggest how to upload if it doesn't
    if not os.path.exists(image_path):
        print(f"Image file 'sample.jpg' not found at {image_path}")
        print("Please upload the 'sample.jpg' file to the 'images' folder")
        return None  # Return None if image is not found

    # Proceed with loading the image
    img = load_img(image_path, target_size=(224, 224))

    # Preprocess the image
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    processed_image = preprocess_input(img_array)

    return processed_image


# Load and preprocess the image using the function
processed_image = load_and_preprocess_image('images/sample.jpg') # Assuming 'sample.jpg' is in the 'images' folder

# Check if image was loaded successfully
if processed_image is not None:
    # Make predictions
    predictions = model.predict(processed_image)

    # Decode and print the predictions
    decoded_predictions = decode_predictions(predictions, top=3)[0]
    print(decoded_predictions)
else:
    print("Image loading failed. Please check the previous messages.")


Created 'images' folder at /content/images
Image file 'sample.jpg' not found at /content/images/sample.jpg
Please upload the 'sample.jpg' file to the 'images' folder
Image loading failed. Please check the previous messages.


In [22]:
# Upload button to load images
upload = widgets.FileUpload()
display(upload)

# Button to make predictions
predict_button = widgets.Button(description="Make Prediction")
display(predict_button)

# Function to handle button click
def on_click(change):
    img_data = list(upload.value.values())[0]['content']
    img = Image.open(io.BytesIO(img_data))
    img = img.resize((224, 224))

    # Preprocess and predict
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    predictions = model.predict(img_array)
    decoded_predictions = decode_predictions(predictions, top=3)[0]

    # Display predictions
    print(decoded_predictions)

predict_button.on_click(on_click)


FileUpload(value={}, description='Upload')

Button(description='Make Prediction', style=ButtonStyle())

# Conclusion and Discussion
Reflect on the lab activities. Discuss how the pre-trained model was able to make predictions, the role of data preprocessing, and the impact of input modifications on the model's predictions.
